In [1]:
# Dependencies
import json
import requests 
from pprint import pprint
import pandas as pd
from pandas import json_normalize
import urllib
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
from datetime import datetime, date
import schedule 
import time

In [2]:
# Specify the URL
url = "https://gbfs.citibikenyc.com/gbfs/es/station_status.json"

# Make request and store response
response = requests.get(url)

# Print status code
print(response.status_code)

200


In [3]:
r = requests.get('https://gbfs.citibikenyc.com/gbfs/en/station_status.json')
df = json_normalize(r.json()['data']['stations'])
df.head()

,station_id,num_bikes_disabled,num_docks_available,num_bikes_available,last_reported,is_renting,eightd_has_available_keys,is_installed,is_returning,num_docks_disabled,...,num_ebikes_available,legacy_id,eightd_active_station_services,valet.off_dock_count,valet.station_id,valet.off_dock_capacity,valet.dock_blocked_count,valet.region,valet.active,valet.valet_revision
0,72,2,50,3,1596495760,1,False,1,1,0,...,0,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,79,0,16,17,1596495698,1,False,1,1,0,...,0,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82,2,1,24,1596494887,1,False,1,1,0,...,0,82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,83,0,29,33,1596495769,1,False,1,1,0,...,0,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,116,2,34,14,1596495789,1,False,1,1,0,...,0,116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df2 = df.rename(columns={"num_bikes_available":"bikes_available","num_bikes_disabled":"bikes_disabled","num_docks_available":"docks_available","num_docks_disabled":"docks_disabled","last_reported":"time_reported"})

In [5]:
df2.head()

,station_id,bikes_disabled,docks_available,bikes_available,time_reported,is_renting,eightd_has_available_keys,is_installed,is_returning,docks_disabled,...,num_ebikes_available,legacy_id,eightd_active_station_services,valet.off_dock_count,valet.station_id,valet.off_dock_capacity,valet.dock_blocked_count,valet.region,valet.active,valet.valet_revision
0,72,2,50,3,1596495760,1,False,1,1,0,...,0,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,79,0,16,17,1596495698,1,False,1,1,0,...,0,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82,2,1,24,1596494887,1,False,1,1,0,...,0,82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,83,0,29,33,1596495769,1,False,1,1,0,...,0,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,116,2,34,14,1596495789,1,False,1,1,0,...,0,116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
bike_df = df2[["station_id","bikes_available","bikes_disabled","docks_available","docks_disabled","time_reported"]]

In [7]:
bike_df.head()

,station_id,bikes_available,bikes_disabled,docks_available,docks_disabled,time_reported
0,72,3,2,50,0,1596495760
1,79,17,0,16,0,1596495698
2,82,24,2,1,0,1596494887
3,83,33,0,29,0,1596495769
4,116,14,2,34,0,1596495789


In [8]:

last_reported = datetime.fromtimestamp(bike_df.loc[0,"time_reported"])

In [9]:
last_reported

datetime.datetime(2020, 8, 3, 16, 2, 40)

In [10]:
def epoch_to_readable(xepoch):
    
    return datetime.fromtimestamp(xepoch)

bike_df['time_reported'] = bike_df.apply(lambda x: epoch_to_readable(x['time_reported']), axis = 1 )

<ipython-input-10-5919d25c1233>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bike_df['time_reported'] = bike_df.apply(lambda x: epoch_to_readable(x['time_reported']), axis = 1 )


In [11]:
bike_df.head()

,station_id,bikes_available,bikes_disabled,docks_available,docks_disabled,time_reported
0,72,3,2,50,0,2020-08-03 16:02:40
1,79,17,0,16,0,2020-08-03 16:01:38
2,82,24,2,1,0,2020-08-03 15:48:07
3,83,33,0,29,0,2020-08-03 16:02:49
4,116,14,2,34,0,2020-08-03 16:03:09


In [12]:
db_connection_string = "postgres:jaigurudev@localhost:5432/FinalProject"
engine = create_engine(f'postgresql://{db_connection_string}')

In [13]:
engine.table_names()

['bikes']

In [14]:
bike_df.to_sql(name = 'bikes',con = engine , if_exists = 'append',index = False)

In [15]:
pd.read_sql_query('select * from bikes', con=engine).head()

,station_id,bikes_available,bikes_disabled,docks_available,docks_disabled,time_reported
0,439,16,0,23,0,2020-08-03 12:08:49
1,3840,7,0,17,0,2020-08-03 12:30:20
2,4036,24,1,11,0,2020-08-03 12:07:08
3,278,10,1,8,0,2020-08-03 12:40:39
4,426,16,6,0,30,2020-08-03 13:14:35
